In [2]:
import os
import re
import numpy as np
import pandas as pd
from glob import glob
from IPython.display import display

method_dir = '/media/temp/DATA21/Project_FAST/CODE/FALST/log/CAMELYON_TextBranchLearnable/Slide_random_Patch_kmedoids_20241121_092359'
log_results = {}
for instance_shot_name in os.listdir(method_dir):

    instance_auc_df = pd.DataFrame(index=[42, 43, 44, 45, 46], columns=[1, 2, 4, 8, 16])
    bag_auc_df = pd.DataFrame(index=[42, 43, 44, 45, 46], columns=[1, 2, 4, 8, 16])

    all_log_paths = glob(os.path.join(method_dir, instance_shot_name, "*.txt"))
    assert len(all_log_paths) == 25
    for log_path in all_log_paths:
        match = re.match(r"Seed(\d+)_(\d+)_(\d+)\.txt", os.path.basename(log_path))
        if match:
            seed, bag_shot, instance_shot = map(int, match.groups())
        else:
            raise ValueError
        assert instance_shot == int(instance_shot_name.split('_')[0])
        with open(log_path, 'r') as f:
            log_text = [x.strip() for x in f.readlines()]

        # instance auc
        instance_auc_text = [x for x in log_text if x.startswith('merge_InsAUC')]
        assert len(instance_auc_text) == 1
        match = re.search(r": (\d+\.\d+),", instance_auc_text[0])
        if match:
            instance_auc = float(match.group(1))
        instance_auc_df.loc[seed, bag_shot] = instance_auc

        # bag auc
        bag_auc_text = [x for x in log_text if x.startswith('merge_BagAUC')]
        assert len(bag_auc_text) == 1
        match = re.search(r": (\d+\.\d+),", bag_auc_text[0])
        if match:
            bag_auc = float(match.group(1))
        bag_auc_df.loc[seed, bag_shot] = bag_auc

    instance_auc_numpy = instance_auc_df.to_numpy().astype(np.float32)
    instance_auc_df.loc['Mean'] = instance_auc_numpy.mean(0)
    instance_auc_df.loc['Std'] = instance_auc_numpy.std(0, ddof=1)
    instance_auc_df.loc['Performance'] = [f"{m:.4f}±{s:.4f}" for m, s in zip(instance_auc_df.loc['Mean'].to_list(), instance_auc_df.loc['Std'].to_list())]

    bag_auc_numpy = bag_auc_df.to_numpy().astype(np.float32)
    bag_auc_df.loc['Mean'] = bag_auc_numpy.mean(0)
    bag_auc_df.loc['Std'] = bag_auc_numpy.std(0, ddof=1)
    bag_auc_df.loc['Performance'] = [f"{m:.4f}±{s:.4f}" for m, s in zip(bag_auc_df.loc['Mean'].to_list(), bag_auc_df.loc['Std'].to_list())]
    
    log_results.update({
        instance_shot: {
            'instance_auc': instance_auc_df,
            'bag_auc': bag_auc_df
        }
    })

    print(f'{instance_shot}-shot Instance')
    print('Instance AUC')
    display(instance_auc_df.style.set_table_styles([{'selector': 'th, td', 'props': [('max-width', '200px')]}]))
    print('Bag AUC')
    display(bag_auc_df.style.set_table_styles([{'selector': 'th, td', 'props': [('max-width', '200px')]}]))

4-shot Instance
Instance AUC


,1,2,4,8,16
42,0.584200,0.722500,0.771200,0.784400,0.793400
43,0.661700,0.700500,0.647500,0.797400,0.720000
44,0.743200,0.747200,0.781900,0.793300,0.750100
45,0.766800,0.747800,0.713400,0.730400,0.765100
46,0.733000,0.730000,0.737000,0.763600,0.680600
Mean,0.697780,0.729600,0.730200,0.773820,0.741840
Std,0.074615,0.019611,0.053670,0.027555,0.043287
Performance,0.6978±0.0746,0.7296±0.0196,0.7302±0.0537,0.7738±0.0276,0.7418±0.0433


Bag AUC


,1,2,4,8,16
42,0.587900,0.597200,0.652400,0.675800,0.640600
43,0.608000,0.619100,0.622000,0.590400,0.666800
44,0.608000,0.623000,0.658200,0.601100,0.662500
45,0.491000,0.530800,0.616200,0.625200,0.654600
46,0.524000,0.645300,0.651400,0.657500,0.456600
Mean,0.563780,0.603080,0.640040,0.630000,0.616220
Std,0.053320,0.043860,0.019400,0.036333,0.089786
Performance,0.5638±0.0533,0.6031±0.0439,0.6400±0.0194,0.6300±0.0363,0.6162±0.0898


64-shot Instance
Instance AUC


...

,1,2,4,8,16
42,0.599000,0.495300,0.645300,0.657500,0.668200
43,0.639500,0.610800,0.408200,0.580000,0.497700
44,0.538400,0.604000,0.626600,0.611500,0.452700
45,0.513300,0.634900,0.548400,0.599400,0.616400
46,0.596500,0.609000,0.559200,0.587500,0.445100
Mean,0.577340,0.590800,0.557540,0.607180,0.536020
Std,0.050788,0.054705,0.093354,0.030567,0.100759
Performance,0.5773±0.0508,0.5908±0.0547,0.5575±0.0934,0.6072±0.0306,0.5360±0.1008


16-shot Instance
Instance AUC


,1,2,4,8,16
42,0.621700,0.705000,0.745500,0.727100,0.799500
43,0.694700,0.723100,0.546700,0.689100,0.685200
44,0.782300,0.852100,0.840900,0.781000,0.695900
45,0.770700,0.745400,0.812300,0.696500,0.673400
46,0.746100,0.746800,0.782800,0.726200,0.621400
Mean,0.723100,0.754480,0.745640,0.723980,0.695080
Std,0.065932,0.057237,0.116701,0.036186,0.065001
Performance,0.7231±0.0659,0.7545±0.0572,0.7456±0.1167,0.7240±0.0362,0.6951±0.0650


Bag AUC


,1,2,4,8,16
42,0.585400,0.577100,0.641300,0.672900,0.646700
43,0.622000,0.622300,0.364400,0.634900,0.638100
44,0.529800,0.629500,0.671100,0.644900,0.552400
45,0.594300,0.608700,0.506500,0.624100,0.582900
46,0.578200,0.645600,0.720600,0.628000,0.500700
Mean,0.581940,0.616640,0.580780,0.640960,0.584160
Std,0.033549,0.025800,0.144662,0.019523,0.060821
Performance,0.5819±0.0335,0.6166±0.0258,0.5808±0.1447,0.6410±0.0195,0.5842±0.0608


In [8]:
[f"{m:.4f}±{s:.4f}" for m, s in zip(bag_auc_df.loc['Mean'].to_list(), bag_auc_df.loc['Std'].to_list())]

['0.5991±0.0592',
 '0.6306±0.0287',
 '0.5823±0.1064',
 '0.6534±0.0516',
 '0.5854±0.1029']